## 토큰화
- https://velog.io/@hyewon0309/LM-%ED%95%9C%EA%B5%AD%EC%96%B4-Tokenizer-Python

In [1]:
import re

### 어절 토큰화

In [2]:
def word_tokenizer(s):
  s = re.compile("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]").sub('', s)
  
  result = s.split(' ')
  result = list(filter(None, result))   # 빈 리스트 요소('') 삭제
  return result

### 음절 토큰화

In [3]:
def syllable_tokenizer(s):
    s = re.compile("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]").sub('', s)
    
    result = []
    for c in s:
        result.append(c)
    return result

### 초성, 중성, 종성 토큰화

In [4]:
NO_CHOSUNG = '-'
NO_JOONGSUNG = '#'
NO_JONGSUNG = '*'

CHOSUNGS = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ',
'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
JOONGSUNGS = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ',
'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
JONGSUNGS = [NO_JONGSUNG,  'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ',
'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

N_CHOSUNGS = 19
N_JOONGSUNGS = 21
N_JONGSUNGS = 28

FIRST_HANGUL = 0xAC00 #'가'
LAST_HANGUL = 0xD7A3 #'힣'

In [5]:

def cho_joong_jong_tokenizer(s):
    result = []

    for c in s:
      uni = ord(c) 
      # 완전한 문자인 경우
      if uni >= FIRST_HANGUL and uni <= LAST_HANGUL:
          code = uni - FIRST_HANGUL
          jongsung_index = code % N_JONGSUNGS
          code //= N_JONGSUNGS
          joongsung_index = code % N_JOONGSUNGS
          code //= N_JOONGSUNGS
          chosung_index = code

          result.append(CHOSUNGS[chosung_index])
          result.append(JOONGSUNGS[joongsung_index])
          result.append(JONGSUNGS[jongsung_index])

      # 초성만 있는 경우
      elif c in CHOSUNGS :
        result = [c, NO_JOONGSUNG, NO_JONGSUNG]
      
      # 중성만 있는 경우
      elif c in JOONGSUNGS :
        result = [NO_CHOSUNG, c, NO_JONGSUNG]
      
      # 종성만 있는 경우
      elif c in JONGSUNGS :
        result = [NO_CHOSUNG, NO_JOONGSUNG, c]

      else :
        return c
        
    return ''.join(result)

### 초중성, 종성 토큰화

In [6]:
def chojoong_jong_tokenizer(s):
    result = []
    for c in s:
        if ord(c) < FIRST_HANGUL or ord(c) > LAST_HANGUL :
            # result.append(c)
            # 초성, 중성, 종성만 있는 것이나, 한글이 아닌 것은 무시됨
            pass
        else:
            code = ord(c) - FIRST_HANGUL
            jongsung_index = code % N_JONGSUNGS

            chojoongsung_code = ord(c) - jongsung_index
            chojoongsung = chr(chojoongsung_code)

            result.append(chojoongsung)
            result.append(JONGSUNGS[jongsung_index])

    return ''.join(result)

### word_cjs_tokenizer : 어절로 나눈 단어를 초중종 토큰화

In [7]:
def word_cjs_tokenizer(s) :
  # 정규표현식
  s = re.compile("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]").sub('', s)
  
  words = s.strip().split(' ')

  result = []
  for word in words :
    tokenized_word = cho_joong_jong_tokenizer(word)
    result.append(tokenized_word)

  result = list(filter(None, result))
  return result

### syllable_cjs_tokenizer : 음절로 나눈 단어를 초중종 토큰화

In [8]:
def syllable_cjs_tokenizer(s) :
  # 정규표현식
  s = re.compile("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]").sub('', s)
  
  words = syllable_tokenizer(s)

  result = []
  for word in words :
    tokenized_word = cho_joong_jong_tokenizer(word)
    result.append(tokenized_word)

  result = list(filter(None, result))
  return result